In [ ]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, poisson
import tropycal.tracks as tracks
import pandas as pd
import numpy as np
import datetime as dt
import os
import re
import geopandas as gpd
import geodatasets
from shapely import geometry, wkb, wkt
import multiprocessing as mp
import sys
import threading
from queue import Queue
import xarray as xr
import pathlib
import sqlite3
import pickle

Western Pacific (WP) locations: 5–60°N 100°–180°E

In [ ]:
s = re.compile('^MRI_HPB_m+\d{3}.nc$')
file_dirs = os.listdir('data')
file_dirs
# data_files = [f for f in file_dirs if s.match(f)]
data_files_dirs = [pathlib.Path('data') / pathlib.Path(f) for f in file_dirs if s.match(f)]
# data_files

In [ ]:
test_storms = xr.open_dataset(data_files_dirs[1])
i = 0
test_storms.track_lat[i]
# for lat in test_storms.track_lat[i]:
#     print(lat.values)
test_storms.where((test_storms.track_lat >= 5) & (test_storms.track_lat <= 60) & (test_storms.track_lon >= 100) & (test_storms.track_lon <= 180), drop=True)
# np.isnat(test_storms.track_time)
# t = geometry.LineString(zip(test_storms.track_lon[i], test_storms.track_lat[i]))
# b = t.wkb
# b.hex()
# tb = wkb.loads(b, hex=True)
# tb
# np.datetime_as_string(test_storms.track_time[1][3].values, unit='m')
# data_files_dirs[1].stem
type(test_storms.track_lat[0].values)

In [ ]:
storms_d = mp.Manager().dict()
# storm_geo = mp.Manager().list()
def create_storm_dict_task(storm_i, storms_d, storms, stem):
    time_l = storms.track_time[storm_i].where(~np.isnat(storms.track_time[storm_i]), drop=True).values
    lon_l = storms.track_lon[storm_i].where(~np.isnan(storms.track_lon[storm_i]), drop=True).values
    lat_l = storms.track_lat[storm_i].where(~np.isnan(storms.track_lat[storm_i]), drop=True).values
    pres_l = storms.track_pres[storm_i].where(~np.isnan(storms.track_pres[storm_i]), drop=True).values
    wind_l = storms.track_wind[storm_i].where(~np.isnan(storms.track_wind[storm_i]), drop=True).values
    # for i in range(len(storms.track_time[storm_i])):
    #     # lat = storms.track_lat[storm_i][i]
    #     # lon = storms.track_lon[storm_i][i]
    #     if not np.isnan(storms.track_lon[storm_i][i]):
    #         coords.append((storms.track_lon[storm_i][i], storms.track_lat[storm_i][i]))
    #     else:
    #         break
    if len(lat_l) <= 1:
        # print(f'{stem}_{storm_i} has no valid coordinates')
        return
    storms_d[f'{stem}_{storm_i}'] = {
        # 'time': storms[i]['time'],
        # 'geometry': geometry.LineString((lat, lon) for lat, lon in zip(storms.track_lon[storm_i], storms.track_lat[storm_i]) if (not np.isnan(lat) and not np.isnan(lon))),
        # 'geometry': geometry.LineString(coords),
        # 'featurecla': 'storm',
        # 'scalerank': 1,
        # 'min_zoom': 1,
        'time': time_l,
        'lat': lat_l,
        'lon': lon_l,
        'pres': pres_l,
        'wind': wind_l
        
    }
    # geometry_d.append(geometry.LineString((lat, lon) for lat, lon in zip(storms.track_lon[storm_i], storms.track_lat[storm_i]) if (not np.isnan(lat) and not np.isnan(lon))))
    # print(storm_i)
def generate_storm_dict(stem):
    storms = xr.open_dataset(f'data/{stem}.nc')
    filtered_storms = storms.where((storms.track_lat >= 5) & (storms.track_lat <= 60) & (storms.track_lon >= 100) & (storms.track_lon <= 180), drop=True)
    # print(filtered_storms)
    print(f"{stem}\n", end='')
    for storm_i in range(len(filtered_storms.storm)):
        create_storm_dict_task(storm_i, storms_d, filtered_storms, stem)
    # pool = mp.Pool(mp.cpu_count())
    # pool.starmap(create_storm_dict_task, ((storm_i, storms_d, filtered_storms, stem) for storm_i in range(len(filtered_storms.storm))))
    # pool.close()
    # pool.join()
# processes = []
# for data in data_files_dirs:
#     storms = xr.open_dataset(data)
#     generate_storm_dict(storms, data.stem)

pool = mp.Pool(mp.cpu_count())
pool.starmap(generate_storm_dict, ([data.stem] for data in data_files_dirs))
pool.close()
pool.join()
storms_d = storms_d._getvalue()
    # processes.append(mp.Process(target=generate_geometry, args=(storms, data.stem)))
    # generate_geometry(storms, data.stem)
# for p in processes:
#     p.start()
# for p in processes:
#     p.join()
# storm_geo = storm_geo._getvalue()
# print(storm_geo)
# generate_geometry(test_storms, "test")
# create_geometry(1203, storm_geo, test_storms, "test")

In [ ]:
with open('storms_d.p', 'wb') as f:
    pickle.dump(storms_d, f)


In [ ]:
with open('storms_d.p', 'rb') as f:
    storms_d = pickle.load(f)
storms_d

In [ ]:
# world = gpd.read_file(geodatasets.get_path('naturalearth.land'))
# df = pd.DataFrame.from_dict(storms_d, orient='index')
# roi = gpd.GeoDataFrame(df, geometry='geometry', crs=world.crs)
# roi
# world_roi = pd.concat([world, roi])
# world_roi

In [ ]:
res = 2.25
# res = 1.
world = gpd.read_file(geodatasets.get_path('naturalearth.land'))
storms = storms_d
storms_df = pd.DataFrame.from_dict(storms, orient='index')
lat_max = storms_df['lat'].apply(np.max).max()
lat_min = storms_df['lat'].apply(np.min).min()
lon_max = storms_df['lon'].apply(np.max).max()
lon_min = storms_df['lon'].apply(np.min).min()
time_min = storms_df['time'].apply(np.min).min()
time_max = storms_df['time'].apply(np.max).max()
year_num = (time_max-time_min).days/365.2425
grids = np.zeros((round((lon_max-lon_min)/res)+1, round((lat_max-lat_min)/res)+1))
for i in storms:
    cell_coords = {}
    coords = []
    for lat, lon in zip(storms[i]['lat'], storms[i]['lon']):
            # grids[round((lon-lon_min)/res), round((lat-lat_min)/res)] += 1
            if (round((lon-lon_min)/res), round((lat-lat_min)/res)) not in coords:
                coords.append((round((lon-lon_min)/res), round((lat-lat_min)/res)))
    for i, j in coords:
        grids[i, j] += 1
            
grids = grids/year_num * 0.1
grid_cells = []
X = []
Y = []
C = np.zeros((grids.shape[1], grids.shape[0]))

for i in range(grids.shape[0]):
    X.append(lon_min + (i+0.5)*res)
for i in range(grids.shape[1]):
    Y.append(lat_min + (i+0.5)*res)
for i in range(grids.shape[0]):
    for j in range(grids.shape[1]):
        C[j, i] = grids[i, j]
        
lon_mesh, lat_mesh = np.meshgrid(X, Y)



In [ ]:
norm = plt.matplotlib.colors.LogNorm(vmin=0.1, vmax=C.max())
fig, ax = plt.subplots()
world.plot(color='gray', edgecolor='black', ax=ax)
ax.figure.set_size_inches(20, 15)
ax.set_xlim(lon_min, lon_max)
ax.set_ylim(lat_min, lat_max)
cmap = plt.get_cmap('rainbow', 10)
colors = cmap(np.linspace(0, 1, 256))
colors[0][3] = 0
cmap = plt.matplotlib.colors.ListedColormap(colors)
pcm = ax.pcolormesh(lon_mesh, lat_mesh, C, cmap=cmap, shading='auto', alpha=0.8, norm=norm)
cbar = fig.colorbar(pcm, ax=ax, orientation='vertical',
                    extend='max',
                    ticks=[0,0.1, 1, 10, 20, 30],
                    pad=0.02)
cbar.ax.set_title('[No./10yr]')
cbar.ax.set_yticklabels(['0','0.1', '1', '10', '20', '30'])
plt.tight_layout()

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))
# world.boundary.plot(ax=ax)
# roi.plot(ax=ax, color='red', aspect=1)

In [ ]:
# datas = [Dataset('data/' + f) for f in data_files]

In [ ]:
# for data in datas[:10]:
#     print(np.nanmax(data['track_lat'][:]), np.nanmin(data['track_lat'][:]), np.nanmax(data['track_lon'][:]), np.nanmin(data['track_lon'][:]))

In [ ]:
# l = mp.Lock()
# num_jobs = 0
# def storm_data(storms, data, output):
#     global num_jobs
#     global l
#     for i in range(0, data.dimensions['storm'].size):
#         track_lat_any = np.array(data.variables['track_lat'][i])[
#             (np.array(data.variables['track_lat'][i]) >= 5) &
#             (np.array(data.variables['track_lat'][i]) <= 60) &
#             (~np.isnan(np.array(data.variables['track_lat'][i])))
#         ]
#         track_lon_any = np.array(data.variables['track_lon'][i])[
#             (np.array(data.variables['track_lon'][i]) >= 100) & 
#             (np.array(data.variables['track_lon'][i]) <= 180) &
#             (~np.isnan(np.array(data.variables['track_lon'][i])))
#         ]
#         if len(track_lat_any) != 0 and len(track_lon_any) != 0:
#             # print(data['track_lat'][i])
#             pass
#         else:
#             continue
#         storms[i] = {
#             'time': data.variables['track_time'][i],
#             'lat': data.variables['track_lat'][i],
#             'lon': data.variables['track_lon'][i],
#             'pres': data.variables['track_pres'][i],
#             'wind': data.variables['track_wind'][i]
#         }
#     output.value += 1
#     l.acquire()
#     sys.stdout.write(f'\rProgress: {output.value*100/num_jobs:.2f}% ({output.value} / {num_jobs})')
#     sys.stdout.flush()
#     l.release()
# storms = mp.Manager().dict()
# output = mp.Value('i', 0)
# jobs = []
# for data in datas:
#     j = mp.Process(target=storm_data, args=(storms, data, output), daemon=True)
#     jobs.append(j)
#     num_jobs += 1
#     print(f"Creating jobs: {num_jobs}", end="\r", flush=True)
# print("\nStarting jobs")
# _ = [j.start() for j in jobs]
# _ = [j.join() for j in jobs]

In [ ]:
# print(storms)

In [ ]:
# storms = {}
# for data in datas[:]:
#     for i in range(0, data.dimensions['storm'].size):
#     # for i in range(0, 10):
#         track_lat_any = np.array(data.variables['track_lat'][i])[
#             (np.array(data.variables['track_lat'][i]) >= 5) &
#             (np.array(data.variables['track_lat'][i]) <= 60) &
#             (~np.isnan(np.array(data.variables['track_lat'][i])))
#         ]
#         track_lon_any = np.array(data.variables['track_lon'][i])[
#             (np.array(data.variables['track_lon'][i]) >= 100) & 
#             (np.array(data.variables['track_lon'][i]) <= 180) &
#             (~np.isnan(np.array(data.variables['track_lon'][i])))
#         ]
#         if len(track_lat_any) != 0 and len(track_lon_any) != 0:
#             # print(data['track_lat'][i])
#             pass
#         else:
#             continue
#         storms[i] = {
#             'time': data.variables['track_time'][i],
#             'lat': data.variables['track_lat'][i],
#             'lon': data.variables['track_lon'][i],
#             'pres': data.variables['track_pres'][i],
#             'wind': data.variables['track_wind'][i]
#         }
# # storms

In [ ]:
# world = gpd.read_file(geodatasets.get_path('naturalearth.land'))
# storm_geo = {}
# for i in storms:
#     storm_geo[i] = {
#         # 'time': storms[i]['time'],
#         'geometry': geometry.LineString((lat, lon) for lat, lon in zip(storms[i]['lon'], storms[i]['lat']) if (not np.isnan(lat) and not np.isnan(lon))),
#         'featurecla': 'storm',
#         'scalerank': 1,
#         'min_zoom': 1,
#     }
# df = pd.DataFrame.from_dict(storm_geo, orient='index')
# roi = gpd.GeoDataFrame(df, geometry='geometry', crs=world.crs)
# roi
# world_roi = pd.concat([world, roi])
# world_roi

In [ ]:
# roi.plot()

In [ ]:
# res = 2.25
# lat_max = np.nanmax([np.nanmax(storms[i]['lat']) for i in storms])
# lat_min = np.nanmin([np.nanmin(storms[i]['lat']) for i in storms])
# lon_max = np.nanmax([np.nanmax(storms[i]['lon']) for i in storms])
# lon_min = np.nanmin([np.nanmin(storms[i]['lon']) for i in storms])
# grids = np.zeros((int((lat_max-lat_min)/res)+1, int((lon_max-lon_min)/res)+1))
# for i in storms:
#     cell_coords = {}
#     for lat, lon in zip(storms[i]['lat'], storms[i]['lon']):
#         if (not np.isnan(lat) and not np.isnan(lon)):
#             cell_coords[int((lat-lat_min)/res), int((lon-lon_min)/res)] = 1
#     for coord in cell_coords:
#         grids[coord] += 1

In [ ]:
# grid_cells = []
# for i in range(grids.shape[0]):
#     for j in range(grids.shape[1]):
#         grid_cells.append({
#             'geometry': geometry.Polygon([(lon_min + j*res, lat_min + i*res), (lon_min + j*res, lat_min + (i+1)*res), (lon_min + (j+1)*res, lat_min + (i+1)*res), (lon_min + (j+1)*res, lat_min + i*res)]),
#             'storm_count': grids[i, j]
#         })
# grid_df = pd.DataFrame(grid_cells)
# grid_gdf = gpd.GeoDataFrame(grid_df, geometry='geometry', crs=world.crs)
# ax = world.plot(color='green', edgecolor='red')
# ax.figure.set_size_inches(20, 15)
# ax.set_xlim(lon_min, lon_max)
# ax.set_ylim(lat_min, lat_max)
# grid_gdf.plot(column='storm_count', ax=ax, legend=True, cmap='RdYlBu_r', alpha=0.8)